In [22]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError
from time import sleep
import twitter_credentials
import oauth2 as oauth

In [23]:
consumer=oauth.Consumer(key=twitter_credentials.CONSUMER_KEY,secret=twitter_credentials.CONSUMER_SECRET)
access_token=oauth.Token(key=twitter_credentials.ACCESS_TOKEN,secret=twitter_credentials.ACCESS_TOKEN_SECRET)
client=oauth.Client(consumer,access_token)
print(client)

In [99]:
user = 'realdonaldtrump'


In [100]:
user = user.lower()
output_file = '{}.json'.format(user)
output_file_short = '{}_short.json'.format(user)
compression = zipfile.ZIP_DEFLATED

In [101]:
with open('2017_all_ids.json') as f:
    ids = json.load(f)

print('total ids: {}'.format(len(ids)))


total ids: 2237


In [102]:
ids[0]

'930815626460921856'

In [103]:
all_data = []
start = 0
end = 1
limit = len(ids)
i = math.ceil(limit / 1)
print(i)

2237


In [104]:

for go in range(i):
    print('currently getting tweet {}'.format(start))
    sleep(7)  # needed to prevent hitting API rate limit
    id_batch = ids[start]
    # tweets = api.statuses_lookup(id_batch)
    url=f"https://api.twitter.com/1.1/statuses/show.json?id={id_batch}&tweet_mode=extended"
    response,data=client.request(url)
    start+=1
    tweets=json.loads(data)
    print(tweets["created_at"])
    all_data.append(tweets)

currently getting tweet 0
Wed Nov 15 15:11:53 +0000 2017
currently getting tweet 1
Thu Feb 09 13:19:08 +0000 2017
currently getting tweet 2
Fri Jul 14 08:09:05 +0000 2017
currently getting tweet 3
Fri Jul 14 07:57:33 +0000 2017
currently getting tweet 4
Thu Jun 29 01:51:18 +0000 2017
currently getting tweet 5
Sun Mar 05 11:32:22 +0000 2017
currently getting tweet 6
Tue Jan 03 15:07:41 +0000 2017
currently getting tweet 7
Mon Nov 27 14:04:51 +0000 2017
currently getting tweet 8
Thu Nov 16 02:11:45 +0000 2017
currently getting tweet 9
Sun Sep 03 03:02:31 +0000 2017
currently getting tweet 10
Fri Sep 22 10:28:02 +0000 2017
currently getting tweet 11
Sun May 21 16:47:25 +0000 2017
currently getting tweet 12
Sat Oct 28 12:22:12 +0000 2017
currently getting tweet 13
Mon May 08 22:46:00 +0000 2017
currently getting tweet 14
Thu Mar 16 15:52:22 +0000 2017
currently getting tweet 15
Tue Jan 03 12:51:15 +0000 2017
currently getting tweet 16
Sat Feb 11 13:24:32 +0000 2017
currently getting tweet 

Fri Jun 23 10:07:50 +0000 2017
currently getting tweet 142
Mon Dec 18 22:18:13 +0000 2017
currently getting tweet 143
Tue Apr 11 20:27:35 +0000 2017
currently getting tweet 144
Sun Oct 08 23:58:43 +0000 2017
currently getting tweet 145
Fri May 05 23:22:22 +0000 2017
currently getting tweet 146
Thu Sep 28 14:01:16 +0000 2017
currently getting tweet 147
Sun Nov 12 00:16:05 +0000 2017
currently getting tweet 148
Wed Nov 29 14:49:14 +0000 2017
currently getting tweet 149
Sat Feb 18 13:31:53 +0000 2017
currently getting tweet 150
Fri Jul 14 08:07:29 +0000 2017
currently getting tweet 151
Fri Jul 28 21:00:19 +0000 2017
currently getting tweet 152
Thu May 04 18:07:29 +0000 2017
currently getting tweet 153
Thu Apr 13 13:08:20 +0000 2017
currently getting tweet 154
Sat Dec 16 20:31:43 +0000 2017
currently getting tweet 155
Thu Sep 14 19:31:55 +0000 2017
currently getting tweet 156
Mon Jun 26 12:50:15 +0000 2017
currently getting tweet 157
Sun Jan 22 12:35:09 +0000 2017
currently getting tweet 1

Wed Oct 25 11:27:36 +0000 2017
currently getting tweet 281
Sat Jul 22 14:43:18 +0000 2017
currently getting tweet 282
Sat Sep 30 11:26:16 +0000 2017
currently getting tweet 283
Fri May 12 11:53:48 +0000 2017
currently getting tweet 284
Sun Nov 12 00:18:32 +0000 2017
currently getting tweet 285
Sat Apr 01 15:59:36 +0000 2017
currently getting tweet 286
Fri May 12 11:59:00 +0000 2017
currently getting tweet 287
Sat Jan 14 13:14:13 +0000 2017
currently getting tweet 288
Tue Jan 24 17:04:01 +0000 2017
currently getting tweet 289
Wed Dec 20 21:44:50 +0000 2017
currently getting tweet 290
Wed Nov 08 04:57:40 +0000 2017
currently getting tweet 291
Tue Sep 26 11:14:41 +0000 2017
currently getting tweet 292
Sat Aug 19 20:41:49 +0000 2017
currently getting tweet 293
Fri Sep 15 10:42:55 +0000 2017
currently getting tweet 294
Fri Sep 15 20:54:52 +0000 2017
currently getting tweet 295
Wed Jan 25 12:13:46 +0000 2017
currently getting tweet 296
Thu Apr 20 19:33:46 +0000 2017
currently getting tweet 2

Wed Jan 25 22:05:59 +0000 2017
currently getting tweet 420
Fri Oct 13 09:36:54 +0000 2017
currently getting tweet 421
Sat Feb 25 22:02:22 +0000 2017
currently getting tweet 422
Mon May 29 12:36:37 +0000 2017
currently getting tweet 423
Sun Feb 05 20:39:05 +0000 2017
currently getting tweet 424
Thu Sep 28 23:41:58 +0000 2017
currently getting tweet 425
Wed Jul 19 00:53:12 +0000 2017
currently getting tweet 426
Thu Sep 07 00:04:12 +0000 2017
currently getting tweet 427
Tue Aug 22 03:50:09 +0000 2017
currently getting tweet 428
Fri Feb 24 12:36:34 +0000 2017
currently getting tweet 429
Mon Jan 30 13:43:25 +0000 2017
currently getting tweet 430
Fri Feb 10 23:24:51 +0000 2017
currently getting tweet 431
Tue Mar 07 13:41:58 +0000 2017
currently getting tweet 432
Fri Sep 22 11:26:06 +0000 2017
currently getting tweet 433
Thu Sep 07 12:33:12 +0000 2017
currently getting tweet 434
Wed Jul 05 02:19:23 +0000 2017
currently getting tweet 435
Sun Sep 03 11:39:38 +0000 2017
currently getting tweet 4

Wed Sep 27 11:36:21 +0000 2017
currently getting tweet 559
Sat Jan 07 00:07:09 +0000 2017
currently getting tweet 560
Mon Feb 13 16:59:06 +0000 2017
currently getting tweet 561
Sat Aug 05 16:00:59 +0000 2017
currently getting tweet 562
Thu May 04 11:02:54 +0000 2017
currently getting tweet 563
Thu Apr 13 00:32:25 +0000 2017
currently getting tweet 564
Sat Mar 25 13:29:17 +0000 2017
currently getting tweet 565
Thu May 04 15:26:00 +0000 2017
currently getting tweet 566
Sat Dec 16 20:05:05 +0000 2017
currently getting tweet 567
Thu Sep 07 19:48:47 +0000 2017
currently getting tweet 568
Fri Nov 03 10:57:47 +0000 2017
currently getting tweet 569
Sat Jul 22 11:35:34 +0000 2017
currently getting tweet 570
Tue Sep 26 10:35:43 +0000 2017
currently getting tweet 571
Fri Oct 20 05:12:44 +0000 2017
currently getting tweet 572
Mon Aug 14 10:38:17 +0000 2017
currently getting tweet 573
Fri Mar 03 20:48:21 +0000 2017
currently getting tweet 574
Sun Mar 05 17:30:11 +0000 2017
currently getting tweet 5

...

Sun Jun 18 10:38:51 +0000 2017
currently getting tweet 1661
Wed Mar 15 10:55:30 +0000 2017
currently getting tweet 1662
Sat Sep 16 23:04:03 +0000 2017
currently getting tweet 1663
Sat Dec 23 22:57:50 +0000 2017
currently getting tweet 1664
Thu Nov 09 13:58:32 +0000 2017
currently getting tweet 1665
Sat Apr 08 19:58:32 +0000 2017
currently getting tweet 1666
Wed Nov 08 03:09:37 +0000 2017
currently getting tweet 1667
Sun Apr 23 15:44:59 +0000 2017
currently getting tweet 1668
Fri Jun 16 13:07:55 +0000 2017
currently getting tweet 1669
Thu Nov 30 19:26:42 +0000 2017
currently getting tweet 1670
Sat Feb 18 00:02:58 +0000 2017
currently getting tweet 1671
Wed Aug 16 21:35:16 +0000 2017
currently getting tweet 1672
Thu Jan 05 13:25:30 +0000 2017
currently getting tweet 1673
Sat Aug 05 23:14:18 +0000 2017
currently getting tweet 1674
Thu Jan 19 01:54:13 +0000 2017
currently getting tweet 1675
Wed Sep 20 12:16:22 +0000 2017
currently getting tweet 1676
Fri Mar 24 17:03:46 +0000 2017
currently

Fri Nov 03 11:55:29 +0000 2017
currently getting tweet 1798
Tue Feb 07 16:25:13 +0000 2017
currently getting tweet 1799
Sun Oct 08 13:59:58 +0000 2017
currently getting tweet 1800
Sat Mar 11 14:39:07 +0000 2017
currently getting tweet 1801
Tue May 16 16:02:17 +0000 2017
currently getting tweet 1802
Tue Dec 19 11:04:11 +0000 2017
currently getting tweet 1803
Mon Jul 10 10:40:16 +0000 2017
currently getting tweet 1804
Tue Jan 17 14:46:27 +0000 2017
currently getting tweet 1805
Tue Oct 31 19:14:31 +0000 2017
currently getting tweet 1806
Tue Mar 28 01:41:15 +0000 2017
currently getting tweet 1807
Wed Sep 27 10:56:53 +0000 2017
currently getting tweet 1808
Sat Sep 30 11:48:36 +0000 2017
currently getting tweet 1809
Tue Oct 31 14:10:42 +0000 2017
currently getting tweet 1810
Sun Dec 24 13:48:11 +0000 2017
currently getting tweet 1811
Fri Mar 31 11:04:32 +0000 2017
currently getting tweet 1812
Wed Jan 04 18:52:09 +0000 2017
currently getting tweet 1813
Fri Nov 24 12:10:46 +0000 2017
currently

Mon Jul 31 12:28:00 +0000 2017
currently getting tweet 1935
Fri Jun 30 20:55:44 +0000 2017
currently getting tweet 1936
Mon Sep 11 18:35:59 +0000 2017
currently getting tweet 1937
Mon Jul 31 22:19:11 +0000 2017
currently getting tweet 1938
Sun Dec 03 02:06:54 +0000 2017
currently getting tweet 1939
Wed Aug 23 22:41:19 +0000 2017
currently getting tweet 1940
Sun Jan 22 14:23:17 +0000 2017
currently getting tweet 1941
Fri Dec 22 14:17:04 +0000 2017
currently getting tweet 1942
Sun Sep 03 11:30:41 +0000 2017
currently getting tweet 1943
Wed Aug 23 13:50:52 +0000 2017
currently getting tweet 1944
Sun Mar 05 11:40:20 +0000 2017
currently getting tweet 1945
Tue Jan 31 11:21:52 +0000 2017
currently getting tweet 1946
Fri Oct 27 21:17:31 +0000 2017
currently getting tweet 1947
Tue May 23 15:58:37 +0000 2017
currently getting tweet 1948
Tue Oct 17 18:59:43 +0000 2017
currently getting tweet 1949
Fri Dec 22 22:11:56 +0000 2017
currently getting tweet 1950
Thu Jun 15 21:43:14 +0000 2017
currently

Thu Nov 09 23:39:56 +0000 2017
currently getting tweet 2072
Mon Dec 18 19:35:38 +0000 2017
currently getting tweet 2073
Thu Dec 28 15:24:53 +0000 2017
currently getting tweet 2074
Sat Dec 09 12:41:39 +0000 2017
currently getting tweet 2075
Sat May 13 17:54:31 +0000 2017
currently getting tweet 2076
Wed Oct 18 13:41:10 +0000 2017
currently getting tweet 2077
Tue Nov 21 22:49:40 +0000 2017
currently getting tweet 2078
Tue Jan 03 18:44:47 +0000 2017
currently getting tweet 2079
Sat Apr 22 19:01:59 +0000 2017
currently getting tweet 2080
Wed Oct 18 20:42:05 +0000 2017
currently getting tweet 2081
Tue Nov 14 09:11:51 +0000 2017
currently getting tweet 2082
Tue Apr 11 12:03:43 +0000 2017
currently getting tweet 2083
Wed Dec 20 14:32:51 +0000 2017
currently getting tweet 2084
Sat Jul 22 10:45:05 +0000 2017
currently getting tweet 2085
Fri Sep 08 12:41:32 +0000 2017
currently getting tweet 2086
Fri Mar 17 15:39:10 +0000 2017
currently getting tweet 2087
Wed May 10 02:42:52 +0000 2017
currently

Thu Dec 21 14:52:22 +0000 2017
currently getting tweet 2209
Wed Nov 08 15:40:40 +0000 2017
currently getting tweet 2210
Sat Jul 01 13:07:05 +0000 2017
currently getting tweet 2211
Sat Oct 07 14:04:39 +0000 2017
currently getting tweet 2212
Tue Aug 15 03:06:52 +0000 2017
currently getting tweet 2213
Tue Mar 28 20:57:17 +0000 2017
currently getting tweet 2214
Wed Mar 08 11:13:44 +0000 2017
currently getting tweet 2215
Thu Apr 06 18:30:49 +0000 2017
currently getting tweet 2216
Sat Feb 04 13:12:02 +0000 2017
currently getting tweet 2217
Thu Oct 26 03:33:41 +0000 2017
currently getting tweet 2218
Fri Oct 20 18:50:21 +0000 2017
currently getting tweet 2219
Wed Oct 11 01:15:26 +0000 2017
currently getting tweet 2220
Tue Sep 12 18:41:50 +0000 2017
currently getting tweet 2221
Fri Dec 01 11:21:06 +0000 2017
currently getting tweet 2222
Wed Mar 01 13:38:44 +0000 2017
currently getting tweet 2223
Fri May 12 21:16:56 +0000 2017
currently getting tweet 2224
Tue Sep 26 01:24:10 +0000 2017
currently

In [105]:
print('metadata collection complete')
print('creating master json file')
with open(output_file, 'w') as outfile:
    json.dump(all_data, outfile)

metadata collection complete
creating master json file


In [106]:
results=[]

In [107]:
print('creating ziped master json file')
zf = zipfile.ZipFile('{}.zip'.format(user), mode='w')
zf.write(output_file, compress_type=compression)
zf.close()

creating ziped master json file


In [108]:
def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

with open(output_file) as json_data:
    data = json.load(json_data)
    for entry in data:
        t = {
            "created_at": entry["created_at"],
            "full_text": entry["full_text"],
            "in_reply_to_screen_name": entry["in_reply_to_screen_name"],
            "retweet_count": entry["retweet_count"],
            "favorite_count": entry["favorite_count"],
            "source": get_source(entry),
            "id_str": entry["id_str"],
            "is_retweet": is_retweet(entry)
        }
        results.append(t)

In [109]:
print('creating minimized json master file')
with open(output_file_short, 'w') as outfile:
    json.dump(results, outfile)

with open(output_file_short) as master_file:
    data = json.load(master_file)
    fields = ["favorite_count", "source", "full_text", "in_reply_to_screen_name", "is_retweet", "created_at", "retweet_count", "id_str"]
   

creating minimized json master file
creating CSV version of minimized json master file


UnicodeEncodeError: 'charmap' codec can't encode characters in position 76-77: character maps to <undefined>